<a href="https://colab.research.google.com/github/ducha-1504/AISI-project/blob/main/Duc's_Yahoo_Finance_Code_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount G-drive to save your output
# It will promp you to give permission to Google to access your file

from google.colab import drive
drive.mount('/content/drive')

#Pandas is a Python library used for working with data sets.

import pandas as pd
import numpy as np

Mounted at /content/drive


In [ ]:
import http.client
import json
import datetime
import csv
import os
import yfinance as yf

In [ ]:
# Perplexity API setup
API_HOST = "api.perplexity.ai"
API_ENDPOINT = "/chat/completions"
API_KEY = "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"

In [ ]:
# Standard headers
CSV_HEADER = ["Date", "Ticker", "Company Name", "Sector", "Weight", "Price Buy", "Price Sell"]

# Prompts dictionary (unchanged)
prompts = {
    "Market_Direction_Prediction": "Today is #TODAY#. Predict the S&P 500 direction for tomorrow. Provide output in CSV:\nDate,Prediction,Confidence",
    "Low_Risk_Portfolio": "Today is #TODAY#. Pretend to be a financial expert. Construct a low-risk portfolio for the S&P 500. "
                          "Provide output in CSV format:\nDate,Ticker,Company Name,Sector,Weight,Price Buy,Price Sell",
    "High_Risk_Portfolio": "Today is #TODAY#. Construct a high-risk portfolio for the S&P 500. "
                           "Provide output in CSV format:\nDate,Ticker,Company Name,Sector,Weight,Price Buy,Price Sell",
    "No_Risk_Specification_Portfolio": "Today is #TODAY#. Construct a portfolio designed to outperform the S&P 500. "
                                       "Provide output in CSV format:\nDate,Ticker,Company Name,Sector,Weight,Price Buy,Price Sell",
    "Value_Based_Investing_Portfolio": "Today is #TODAY#. Use a value investing strategy to pick 25 S&P 500 stocks. "
                                       "Provide output in CSV format:\nDate,Ticker,Company Name,Sector,Weight,Price Buy,Price Sell",
    "Growth_Based_Investing_Portfolio": "Today is #TODAY#. Use a growth investing strategy to pick 25 S&P 500 stocks. "
                                        "Provide output in CSV format:\nDate,Ticker,Company Name,Sector,Weight,Price Buy,Price Sell",
    "Dividend_Based_Investing_Portfolio": "Today is #TODAY#. Use a dividend investing strategy to pick 25 S&P 500 stocks. "
                                          "Provide output in CSV format:\nDate,Ticker,Company Name,Sector,Weight,Price Buy,Price Sell"
}


In [ ]:
# Query Perplexity API (unchanged)
def query_perplexity(prompt):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "r1-1776",
        "messages": [
            {"role": "system", "content": "You are a financial AI expert."},
            {"role": "user", "content": prompt}
        ]
    }

    connection = http.client.HTTPSConnection(API_HOST)
    connection.request("POST", API_ENDPOINT, body=json.dumps(payload), headers=headers)
    response = connection.getresponse()

    if response.status == 200:
        return response.read().decode("utf-8").strip()
    else:
        print(f"Error {response.status}")
        return None

In [ ]:
# Extract CSV from response
def extract_csv(response_text):
    try:
        data = json.loads(response_text)
        content = data["choices"][0]["message"]["content"]

        csv_start = content.find("```csv")
        csv_end = content.rfind("```")

        if csv_start != -1 and csv_end != -1:
            return content[csv_start + len("```csv"):csv_end].strip()
        else:
            return None
    except (json.JSONDecodeError, KeyError, IndexError) as e:
        print(f"Error parsing response: {e}")
        return None

In [ ]:
def clean_row(row):
    fields = row.split(',')
    if len(fields) < 7:
        return None
    try:
        weight_val = float(fields[4].strip().replace('%', ''))
        price_buy_val = float(fields[5].strip().replace('$', ''))
        price_sell_val = float(fields[6].strip().replace('$', ''))

        weight_formatted = f"{round(weight_val, 2):.2f}%"
        price_buy_formatted = f"${round(price_buy_val, 2):.2f}"
        price_sell_formatted = f"${round(price_sell_val, 2):.2f}"

        return fields[:4] + [weight_formatted, price_buy_formatted, price_sell_formatted]
    except:
        return None

In [ ]:
def update_latest_10_prices_buy(filename, num_rows=10):
    try:
        df = pd.read_csv(filename)

        # Only process if we have at least 10 rows
        if df.shape[0] < num_rows:
            return

        latest_df = df.tail(num_rows).copy()
        tickers = latest_df['Ticker'].unique().tolist()

        # Fetch latest Close prices for tickers
        prices = yf.download(tickers=tickers, period="1d", interval="1d", progress=False)["Close"]
        if isinstance(prices, pd.Series):  # If single ticker, convert to dict
            prices = prices.to_dict()
        else:
            prices = prices.iloc[-1].to_dict()  # Last available row

        # Update latest 10 rows in the main dataframe
        for i in latest_df.index:
            ticker = df.loc[i, 'Ticker']
            if ticker in prices:
                price = round(float(prices[ticker]), 2)
                df.at[i, 'Price Buy'] = f"${price:.2f}"

        df.to_csv(filename, index=False)
        print(f"✅ Updated real-time prices in: {filename}")

    except Exception as e:
        print(f"Error updating prices in {filename}: {e}")

In [ ]:
def update_latest_10_price_sell(filename, num_rows=10):
    try:
        df = pd.read_csv(filename)
        if df.shape[0] < num_rows:
            return

        latest_df = df.tail(num_rows).copy()
        tickers = latest_df['Ticker'].unique().tolist()

        # Fetch the latest High prices
        highs = yf.download(tickers=tickers, period="1d", interval="1d", progress=False)["High"]

        if isinstance(highs, pd.Series):
            highs = highs.to_dict()
        else:
            highs = highs.iloc[-1].to_dict()

        for i in latest_df.index:
            ticker = df.loc[i, 'Ticker']
            if ticker in highs:
                price = round(float(highs[ticker]), 2)
                df.at[i, 'Price Sell'] = f"${price:.2f}"

        df.to_csv(filename, index=False)
        print(f"✅ Updated real-time 'Price Sell' in: {filename}")

    except Exception as e:
        print(f"Error updating Price Sell in {filename}: {e}")


In [ ]:
def append_to_csv(csv_data, filename):
    if not csv_data:
        print(f"No CSV data to write for {filename}")
        return

    file_exists = os.path.exists(filename)

    lines = csv_data.strip().split('\n')
    header = lines[0]
    rows = lines[1:26]
    temp_weights = []
    temp_data = []

    for row in rows:
        cleaned = clean_row(row)
        if cleaned:
            weight_val = float(cleaned[4].replace('%', ''))
            temp_weights.append(weight_val)
            temp_data.append(cleaned)

    total_weight = sum(temp_weights)

    normalized_rows = []
    for i, row in enumerate(temp_data):
      new_weight = (temp_weights[i] / total_weight) * 100
      weight_str = f"{round(new_weight, 2):.2f}%"
      normalized_rows.append(row[:4] + [weight_str, row[5], row[6]])

    try:
        with open(filename, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            if not file_exists:
                writer.writerow(header.split(','))
            writer.writerows(normalized_rows)
        print(f" Data appended to {filename}")

    except Exception as e:
        print(f"Error writing to {filename}: {e}")

In [ ]:
import time
def main():
    today = datetime.date.today().strftime("%Y-%m-%d")
    output_dir = '/content/drive/My Drive/Finance_Portfolios'
    os.makedirs(output_dir, exist_ok=True)

    successful_files = []

    for key, raw_prompt in prompts.items():
        formatted_prompt = raw_prompt.replace("#TODAY#", today)
        response = query_perplexity(formatted_prompt)

        if response:
            csv_text = extract_csv(response)
            if csv_text:
                filename = f"/content/drive/My Drive/Finance_Portfolios/{key}.csv"
                append_to_csv(csv_text, filename)
            else:
                print(f"Failed to extract CSV for {key}")
        else:
            print(f"API call failed for {key}")
    print("\n⏳ Updating real-time prices...\n")
    target_file = "/content/drive/My Drive/Finance_Portfolios/No_Risk_Specification_Portfolio.csv"
    if os.path.exists(target_file):
          update_latest_10_prices_buy(target_file, num_rows=10)  # For Price Buy
          update_latest_10_price_sell(target_file, num_rows=10)  # For Price Sell
    else:
      print("⚠️ Target portfolio file not found.")


if __name__ == "__main__":
    main()


 Data appended to /content/drive/My Drive/Finance_Portfolios/Market_Direction_Prediction.csv
 Data appended to /content/drive/My Drive/Finance_Portfolios/Low_Risk_Portfolio.csv
 Data appended to /content/drive/My Drive/Finance_Portfolios/High_Risk_Portfolio.csv
 Data appended to /content/drive/My Drive/Finance_Portfolios/No_Risk_Specification_Portfolio.csv
 Data appended to /content/drive/My Drive/Finance_Portfolios/Value_Based_Investing_Portfolio.csv
Failed to extract CSV for Growth_Based_Investing_Portfolio
 Data appended to /content/drive/My Drive/Finance_Portfolios/Dividend_Based_Investing_Portfolio.csv

⏳ Updating real-time prices...



/tmp/ipython-input-319915760.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(tickers=tickers, period="1d", interval="1d", progress=False)["Close"]


✅ Updated real-time prices in: /content/drive/My Drive/Finance_Portfolios/No_Risk_Specification_Portfolio.csv


/tmp/ipython-input-4241283881.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  highs = yf.download(tickers=tickers, period="1d", interval="1d", progress=False)["High"]


✅ Updated real-time 'Price Sell' in: /content/drive/My Drive/Finance_Portfolios/No_Risk_Specification_Portfolio.csv
